In [1]:
import pandas as pd
import editdistance
import numpy as np

# Read Excel file and iterate over each sheet
excel_file = "cdc_2021.xlsx"
sheet_names = pd.ExcelFile(excel_file).sheet_names

dfs = {}  # Dictionary to store dataframes for each sheet
for sheet_name in sheet_names:
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    df.drop_duplicates(inplace=True)
    df = df[['RegNo', 'Name', "FatherName", "MotherName", "DOB"]]
    df["Name"] = df["Name"].str.strip().str.title().sort_values(ascending=True)
    df["FatherName"] = df["FatherName"].str.strip().str.title().sort_values(ascending=True)
    df["MotherName"] = df["MotherName"].str.strip().str.title().sort_values(ascending=True)
    df["DOB"] = df["DOB"].astype(str)
    df.dropna(subset=['Name', 'FatherName', 'MotherName'], inplace=True)

    print(f"Dataframe: {df.shape}")
    
    grouped_df = df.groupby("RegNo").agg({
        "Name": lambda x: '|'.join(x.unique()),
        "DOB": lambda x: '|'.join(x.unique()),
        "FatherName": lambda x: '|'.join(x.unique()),
        "MotherName": lambda x: '|'.join(x.unique())
    }).reset_index()
    
    grouped_df["Unique_Count_Name"] = df.groupby("RegNo")["Name"].nunique().values
    grouped_df["Unique_Count_DOB"] = df.groupby("RegNo")["DOB"].nunique().values
    grouped_df["Unique_Count_FatherName"] = df.groupby("RegNo")["FatherName"].nunique().values
    grouped_df["Unique_Count_MotherName"] = df.groupby("RegNo")["MotherName"].nunique().values

    grouped_df["Name"] = grouped_df["Name"].astype(str)
    grouped_df["DOB"] = grouped_df["DOB"].astype(str)
    grouped_df["FatherName"] = grouped_df["FatherName"].astype(str)
    grouped_df["MotherName"] = grouped_df["MotherName"].astype(str)
    
    print("Grouped Data", grouped_df.shape)

    for col in ["Name", "FatherName", "MotherName", "DOB"]:
        max_count = grouped_df[f"Unique_Count_{col}"].max()
        for i in range(1, max_count + 1):
            grouped_df[f"{col}{i}"] = grouped_df[col].str.split('|', expand=True)[i - 1]
            # if grouped_df[f"Unique_Count_{col}"].nunique() == 1 or i > grouped_df[f"Unique_Count_{col}"].nunique():
            #     grouped_df.loc[:, f'{col}{i}':] = None
        grouped_df[f"Flag_{col}"] = 0
    
    grouped_df.drop(["Name", "FatherName", "MotherName", "DOB"], axis=1, inplace=True)
    
    dfs[sheet_name] = grouped_df

# print("DFS" , dfs["One"].shape)
# print("DFS" , dfs["Two"].shape)
# print("DFS" , dfs["Three"].shape)
# print("DFS" , dfs["Four"].shape)


# Now you can access each dataframe using sheet names as keys in the dfs dictionary
print(dfs.keys())  # To see the names of dataframes

Dataframe: (314540, 5)
Grouped Data (309997, 9)
dict_keys(['Sheet1'])


In [2]:
import pandas as pd
import editdistance

# Function to calculate character changes between two names
def calculate_character_changes(**kwargs):
    original_name = kwargs.get('original_name', '')
    mismatched_name = kwargs.get('mismatched_name', '')
    col_name = kwargs.get('col_name', '')

    if mismatched_name is None:
        mismatched_name = original_name

    original_words = sorted(original_name.strip().split())
    mismatched_words = sorted(mismatched_name.strip().split())
    sorted_original = ' '.join(original_words)
    sorted_mismatched = ' '.join(mismatched_words)
    sorted_distance = editdistance.eval(sorted_original, sorted_mismatched)
    modified_distance = editdistance.eval(original_name.replace(" ", ""), mismatched_name.replace(" ", ""))
    max_distance = max(len(original_name), len(mismatched_name))

    entire_mismatch = 0

    if set(sorted_original) != set(sorted_mismatched):
        entire_mismatch = 1  # Set entire mismatch flag to 1
        # print(f"{sorted_original},{sorted_mismatched}, {entire_mismatch}")

    if sorted_distance == 0:
        modified_distance = 0
        max_distance = 0
    else:
        if sorted_original != sorted_mismatched:
            max_distance = max(len(original_name), len(mismatched_name))
        if modified_distance < sorted_distance:
            sorted_distance = modified_distance
            
    return sorted_distance, entire_mismatch, col_name

# Iterate over each dataframe in dfs dictionary
for sheet_name, df in dfs.items():
    for col in ["Name", "FatherName", "MotherName", "DOB"]:
        max_count = df[f"Unique_Count_{col}"].max()
        
        # Iterate over the columns associated with the current name category
        for i in range(2, max_count + 1):
            mismatched_col_name = f"{col}{i}"
            
            # Ensure the column exists in the dataframe
            if mismatched_col_name in df.columns:
                original_col_name = f"{col}1"
                original_values = df[original_col_name]
                mismatched_values = df[mismatched_col_name]
                
                # Calculate character differences for each pair of original and mismatched values
                sorted_distances = []
                entire_mismatches = []
                
                for original_value, mismatched_value in zip(original_values, mismatched_values):
                    sorted_distance, entire_mismatch, _ = calculate_character_changes(
                        original_name=original_value,
                        mismatched_name=mismatched_value,
                        col_name=mismatched_col_name
                    )
                    sorted_distances.append(sorted_distance)
                    entire_mismatches.append(entire_mismatch)
                
                # Add columns for character differences
                df[f"{col}_Char_Sorted_Diff_{i}"] = sorted_distances
                df[f"{col}_Entire_Mismatch_{i}"] = entire_mismatches
                
    # Update the dataframe in the dfs dictionary
    dfs[sheet_name] = df


In [3]:
dfs1 = dfs.copy()

In [4]:
# dfs["One"].columns

In [5]:
# dfs["Four"]['Name_Entire_Mismatch_2'].value_counts()

In [6]:
for sheet_name, df in dfs.items():
    for col in ["Name", "FatherName", "MotherName", "DOB"]:
        max_count = df[f"Unique_Count_{col}"].max()

        # Initialize the flag column
        df[f"Flag_{col}"] = 0

        # Iterate over the columns associated with the current name category
        for i in range(2, max_count + 1):
            sorted_diff_col_name = f"{col}_Char_Sorted_Diff_{i}"
            
            
            # Ensure the column exists in the dataframe
            if sorted_diff_col_name in df.columns:
                # Update the flag column based on the character differences
                df[f"Flag_{col}"] |= df[sorted_diff_col_name].apply(lambda x: 1 if x >= 1 else 0)
    
    # Update the dataframe in the dfs dictionary
    dfs[sheet_name] = df


In [7]:
dfs2 = dfs.copy()

In [8]:
# dfs["Four"]["Flag_Name"].value_counts()

In [9]:
for sheet_name, df in dfs.items():
    final_status_sum = 0  # Initialize sum for Final_Status
    
    for col in ["Name", "FatherName", "MotherName", "DOB"]:
        final_status_sum += df[f"Flag_{col}"]  # Accumulate sum of Flag columns
        
    # Assign the total sum to the "Final_Status" column
    df["Final_Status"] = final_status_sum
    
    # Update the dataframe in the dfs dictionary
    dfs[sheet_name] = df


In [10]:
dfs3 = dfs.copy()

In [11]:
# dfs["One"]["Final_Status"].value_counts()

In [12]:
import pandas as pd

# Assuming dfs is your dictionary of DataFrames
for sheet_name, df in dfs.items():
    new_dfs = []  # List to store reordered DataFrames for each column
    
    # Extract 'RegNo' column once
    regno_df = df[['RegNo']]
    final_df = df[["Final_Status"]]    
    for col in ["Name", "FatherName", "MotherName", "DOB"]:
        # Check if the column exists in df
        if f"Unique_Count_{col}" in df.columns:
            max_count = df[f"Unique_Count_{col}"].max()
            
            # Define the desired order of columns
            columns_order = [f'{col}{i}' for i in range(1, max_count + 1)] + \
                            [f'{col}_Char_Sorted_Diff_{i}' for i in range(2, max_count + 1)] + \
                            [f'{col}_Entire_Mismatch_{i}' for i in range(2, max_count + 1)] + \
                            [f'Unique_Count_{col}'] + \
                            [f'Flag_{col}' ] 
            
            # Reorder the columns using loc
            reordered_df = df.loc[:, columns_order]
            
            # Append the reordered DataFrame to the list
            new_dfs.append(reordered_df)
    
    # Concatenate all reordered DataFrames along the columns axis
    if new_dfs:
        concatenated_df = pd.concat(new_dfs, axis=1)
        new_df = pd.concat([regno_df, concatenated_df, final_df], axis=1)
    else:
        new_df = df
    
    # Update the dataframe in the dfs dictionary
    dfs[sheet_name] = new_df


In [13]:
# dfs["One"].columns

In [14]:
# Specify the output Excel file path
output_excel_file = "cdc_2021_data_analysis.xlsx"

# Create a Pandas Excel writer object
with pd.ExcelWriter(output_excel_file) as writer:
    # Iterate over the dictionary of dataframes
    for sheet_name, df in dfs.items():
        
        # Write each dataframe to a separate sheet in the Excel file
        df.to_excel(writer, sheet_name=sheet_name, index=False)

# Confirm that the Excel file has been saved
print(f"Dataframes have been saved to '{output_excel_file}'.")


Dataframes have been saved to 'cdc_2021_data_analysis.xlsx'.
